In [ ]:
import pandas as pd
import datetime
import time
import numpy as np
from sklearn.linear_model import LinearRegression


com_1m_bar = pd.read_parquet('/home/mw/input/com_1m_bar1102/com_30m_bar.parquet', engine = 'fastparquet')
com_1m_bar = com_1m_bar.reset_index()
invalid_code = ['BC', 'CJ', 'EB', 'EG', 'FU', 'I', 'JM', 'LG', 'LH', 'LU', 'NR', 'P',
               'PF', 'PG', 'PK', 'PX', 'RR', 'SA', 'SC', 'SH', 'SP', 'SS', 'UR', 'WR', 
               'PM', 'BB', 'RI', 'JR', 'LR', 'RS', 'WH']
com_1m_bar = com_1m_bar[~com_1m_bar['underlying_symbol'].isin(invalid_code)]
com_1m_bar = com_1m_bar[
    (com_1m_bar['datetime'].dt.time >= datetime.time(9, 0)) & 
    (com_1m_bar['datetime'].dt.time <= datetime.time(15, 0))
]
com_1m_bar = com_1m_bar[com_1m_bar.trading_date >= '2018-01-01']

com_1m_bar["log_close"] = np.log(com_1m_bar["close"])
com_1m_bar["log_ret"]   = com_1m_bar.groupby("underlying_symbol")["log_close"].diff()
pivot_ret = com_1m_bar.pivot(index="datetime", columns="underlying_symbol", values="log_ret").iloc[1:,:]

df = pivot_ret.copy()
df.reset_index(inplace = True)
df['date'] = df['datetime'].dt.date
df_cleaned = df.groupby('date', group_keys=False).apply(lambda group: group.iloc[1:-1])
df_cleaned = df_cleaned.drop(columns=['date'])
df_cleaned.set_index('datetime', inplace = True)

In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np, pandas as pd, random, datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

param_grid = {
    "epochs" : [50, 100],
    "dropout": [0.0, 0.1, 0.2],
    "max_w2" : [10., 100., 1000., np.finfo(np.float32).max],
    "l1"     : [0.0, 1e-5, 1e-4],
    "l2"     : [0.0, 1e-5, 1e-4],
    "rho"    : [0.90, 0.95, 0.99, 0.999],
    "eps"    : [1e-10, 1e-8, 1e-6, 1e-4],
}


def build_super_matrix(df, target, lag=3):
    lagged = [df.shift(i).add_suffix(f"_lag{i}") for i in range(lag)]
    X = pd.concat(lagged, axis=1);   y = df[target].shift(-1)
    m = X.notna().all(1) & y.notna()
    return X[m], y[m], X.index[m]


def _block(sizes, drop):
    layers=[]
    for i in range(len(sizes)-1):
        layers += [nn.Linear(sizes[i], sizes[i+1]),
                   nn.BatchNorm1d(sizes[i+1]),
                   nn.ReLU(inplace=True)]
        if drop>0: layers.append(nn.Dropout(drop))
    return nn.Sequential(*layers)

depth_cfg = {"NN1":[32],
             "NN2":[32,16],
             "NN3":[32,16,8],
             "NN4":[32,16,8,4],
             "NN5":[32,16,8,4,2]}

def make_net(name, in_dim, drop):
    hidden = depth_cfg[name]
    return nn.Sequential(_block([in_dim]+hidden, drop),
                         nn.Linear(hidden[-1],1))


def train_one(model, X, y, X_val, y_val, cfg, batch_size=64):
    crit = nn.SmoothL1Loss(beta=0.999)
    opt = torch.optim.Adadelta(
        model.parameters(),
        rho=cfg["rho"],
        eps=cfg["eps"],
        weight_decay=cfg["l2"]
    )
    best_state = model.state_dict()
    best, wait = np.inf, 0

    # 构建 DataLoader
    dataset = TensorDataset(
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32)
    )
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)  # 注意: shuffle=False

    # 验证集 tensor
    Xv = torch.tensor(X_val, dtype=torch.float32, device=DEVICE)
    yv = torch.tensor(y_val, dtype=torch.float32, device=DEVICE)

    for epoch in range(cfg["epochs"]):
        model.train()
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            pred = model(xb).squeeze(-1)
            loss = crit(pred, yb)
            if cfg["l1"] > 0:
                loss += cfg["l1"] * sum(p.abs().sum() for p in model.parameters())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), cfg["max_w2"])
            opt.step()

        # 验证
        model.eval()
        with torch.no_grad():
            vloss = crit(model(Xv).squeeze(-1), yv).item()

        if vloss < best - 1e-6:
            best, best_state, wait = vloss, model.state_dict(), 0
        else:
            wait += 1
            if wait >= 10:  # early stopping
                break

    model.load_state_dict(best_state)
    return best


def fit_best_model(name, X_tr, y_tr, X_va, y_va,
                   n_iter=25, n_restart=10):
    best, best_cfg = np.inf, None
    for _ in range(n_iter):
        cfg = {k: random.choice(v) for k,v in param_grid.items()}
        model = make_net(name, X_tr.shape[1], cfg["dropout"]).to(DEVICE)
        vloss = train_one(model, X_tr, y_tr, X_va, y_va, cfg)
        if vloss < best:
            best, best_cfg = vloss, cfg

    # —— 用最佳 cfg + train+valid 合并数据，重启 n_restart 次平均 ——
    XY = np.vstack([X_tr, X_va]);  y = np.hstack([y_tr, y_va])
    preds = []
    for seed in range(n_restart):
        torch.manual_seed(seed)
        net = make_net(name, X_tr.shape[1], best_cfg["dropout"]).to(DEVICE)
        train_one(net, XY, y, X_va, y_va, best_cfg)
        with torch.no_grad():
            preds.append(net(torch.tensor(XY, dtype=torch.float32, device=DEVICE))
                         .squeeze(-1).cpu().numpy())
    # 取 ensemble 均值模型（权重=1/n）
    ensemble_pred = np.mean(preds, axis=0)
    return net, ensemble_pred  # net仅作占位；真正用 scaler 反标即可


def month_end(d):
    y,m = d.year,d.month
    return datetime.datetime(y+(m==12),(m%12)+1,1)-datetime.timedelta(days=1)

def rolling_nn(df, target_symbol, model_name="NN1", lag=3,
               start_date=datetime.datetime(2018,1,1),
               end_date  =datetime.datetime(2022,12,31)):

    X_raw, y, idx = build_super_matrix(df, target_symbol, lag)
    data = pd.concat([pd.DataFrame({"y":y.values}, index=idx), X_raw], axis=1)

    preds, cur = [], start_date
    while True:
        tr_end  = month_end(cur + relativedelta(months=2))
        va_end  = month_end(tr_end + relativedelta(months=1))
        te_end  = month_end(va_end + relativedelta(months=1))
        if te_end > end_date: break

        df_tr = data[(data.index>=cur)&(data.index<=tr_end)]
        df_va = data[(data.index>tr_end)&(data.index<=va_end)]
        df_te = data[(data.index>va_end)&(data.index<=te_end)]
        if min(len(df_tr),len(df_va),len(df_te)) < 100:
            cur += relativedelta(months=1); continue

        # X 标准化
        x_scaler = StandardScaler().fit(df_tr[X_raw.columns])
        X_tr,X_va,X_te = (x_scaler.transform(df_[X_raw.columns])
                          for df_ in (df_tr,df_va,df_te))
        # y 标准化 (fit on train only)
        y_mean, y_std = df_tr.y.mean(), df_tr.y.std()
        if y_std < 1e-12: y_std = 1.0
        y_tr = (df_tr.y - y_mean)/y_std
        y_va = (df_va.y - y_mean)/y_std

        # 训练模型
        model, _ = fit_best_model(model_name, X_tr, y_tr, X_va, y_va)

        # 预测并反标准化
        Xte_t = torch.tensor(X_te, dtype=torch.float32, device=DEVICE)
        with torch.no_grad():
            y_pred_std = model(Xte_t).squeeze(-1).cpu().numpy()
        y_pred = y_pred_std * y_std + y_mean

        preds.append(pd.DataFrame({
            "datetime": df_te.index,
            "y_true":   df_te.y.values,
            "y_pred":   y_pred
        }))
        print(f"[{cur:%Y-%m}] test {va_end+datetime.timedelta(days=1):%Y-%m-%d}~{te_end:%Y-%m-%d}")
        cur += relativedelta(months=1)

    return pd.concat(preds, ignore_index=True)

# ───────────────────────────────────────────────────────────
# 6. OOS R²
# ───────────────────────────────────────────────────────────
def r2_os_zero(df):
    sse_m = ((df.y_true-df.y_pred)**2).sum()
    sse_0 = (df.y_true**2).sum()
    return 1 - sse_m/sse_0